# Evaluation using BERTscore

[Source](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [1]:
import pandas as pd

df = pd.read_csv("../agent/predictions.csv")
df = df.drop(columns=['debias_reasoning'])
df = df.rename(columns={"biases": "biases_detected","output":"output_agent"})
df.head()

,input,biases_detected,scores,output_agent,index
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,UNBIASED,0
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Desde hoy y hasta el 19 de diciembre puedes de...,1
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,UNBIASED,2
3,¡Les esperamos!,UNBIASED,1.0,UNBIASED,3
4,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,Estimada Comunidad: La Subdirección de Puebl...,4


In [ ]:
df['output'] = df.apply(lambda row: row['input'] if row['biases_detected'] == 'UNBIASED' else row['output_agent'], axis=1)

In [3]:
df['biases_detected'] = df['biases_detected'].fillna('UNBIASED')
df['output'] = df['output'].fillna('UNBIASED')
len(df)

819

In [4]:
df_causal = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
df = pd.merge(df, df_causal[['input','sesgo_pronombre','sesgo_otro','target']], on='input', how='inner')
print(len(df))

748


In [5]:
df.head()

,input,biases_detected,scores,output_agent,index,output,sesgo_pronombre,sesgo_otro,target
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,UNBIASED,0,Estimada comunidad beauchefiana: ¿Tienes papel...,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Desde hoy y hasta el 19 de diciembre puedes de...,1,Desde hoy y hasta el 19 de diciembre puedes de...,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,UNBIASED,2,Revisa en el afiche qué tipo de papeles puedes...,NaN,NaN,['Revisa en el afiche qué tipo de papeles pued...
3,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,Estimada Comunidad: La Subdirección de Puebl...,4,Estimada Comunidad: La Subdirección de Puebl...,NO,NO,['Estimada Comunidad: La Subdirección de Pue...
4,"Postulaciones, labores y más información en: ...",UNBIASED,1.0,UNBIASED,5,"Postulaciones, labores y más información en: ...",NaN,NaN,"['Postulaciones, labores y más información en:..."


In [6]:
import ast
from collections import Counter

df['target'] = df['target'].apply(lambda text: ast.literal_eval(text))
Counter([len(t) for t in df['target']])

Counter({1: 692, 7: 14, 6: 14, 8: 13, 5: 6, 2: 5, 4: 2, 3: 2})

### Example

There are potentially many options for replacing a biased text. We will consider the highest similarity among all candidate options.

In [7]:
references = ['Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.']
predictions = ['Se informa a toda la comunidad estudiantil que las personas que NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, no podrán obtener esta nueva versión desde el próximo año.' for _ in references]

In [8]:
from evaluate import load
bertscore = load("bertscore")

In [9]:
results = bertscore.compute(predictions=predictions, references=references, model_type="bert-base-multilingual-cased")
print(results)

{'precision': [0.9533587098121643], 'recall': [0.9690830707550049], 'f1': [0.9611566066741943], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.47.1)'}


Full evaluation

Notice that the target either keeps the original text or corrects the bias in one of the several possible ways. We will separate the dataframe in a way that each one of them becomes a separate candidate for the evaluation.

In [10]:
df = df.explode('target', ignore_index=True)

In [11]:
results = bertscore.compute(predictions=df['output'], references=df['target'], model_type="bert-base-multilingual-cased")

In [12]:
df['precision'] = results['precision']
df['recall'] = results['recall']
df['f1'] = results['f1']

In [13]:
len(df)

1032

In [14]:
def filter_best_scores(df, id_column, score_column):
    """
    Reduces the dataframe to only the rows with the best score for each ID.

    Parameters:
    - df (pd.DataFrame): The original dataframe
    - id_column (str): Name of the column containing unique IDs
    - score_column (str): Name of the column containing the scores

    Returns:
    - pd.DataFrame: A filtered dataframe with the best score per ID
    """
    # Find the maximum score for each ID_row
    best_scores = df.groupby(id_column)[score_column].transform('max')
    
    # Filter the rows where the score matches the maximum score for each ID_row
    filtered_df = df[df[score_column] == best_scores].copy(deep=True)
    
    return filtered_df

In [15]:
df_results = filter_best_scores(df, 'index', 'f1')
len(df_results)

748

In [16]:
import numpy as np

for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f'\n\tmean: {np.mean(df_results[k])}\n\tstd: {np.std(df_results[k])}\n')

precision 
	mean: 0.9269657409127383
	std: 0.1404723116821046

recall 
	mean: 0.9108940684460701
	std: 0.16598013810188506

f1 
	mean: 0.9183074265399719
	std: 0.1538533557165024



Subtracting input

In [17]:
results_input = bertscore.compute(
    predictions=df_results['input'],
    references=df_results['target'], model_type="bert-base-multilingual-cased")
df_results['precision_input'] = results_input['precision']
df_results['recall_input'] = results_input['recall']
df_results['f1_input'] = results_input['f1']

df_results['precision_diff'] = df_results['precision'] - df_results['precision_input']
df_results['recall_diff'] = df_results['recall'] - df_results['recall_input']
df_results['f1_diff'] = df_results['f1'] - df_results['f1_input']

In [18]:
for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f"\n\tmean (diff): {np.mean(df_results[k+'_diff'])}\n\tstd (diff): {np.std(df_results[k+'_diff'])}\n")

precision 
	mean (diff): -0.0718283454842746
	std (diff): 0.14032722737511602

recall 
	mean (diff): -0.0870099298377088
	std (diff): 0.16570363491985357

f1 
	mean (diff): -0.08003377404442445
	std (diff): 0.15364305554792193

